In [2]:
import sys, os
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
print(os.getcwd())

import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag_gumbel_softmax import train_gumbel_sgd, train_gs_reset
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm

from SCM_data import generate_scm_from_BN 


c:\Users\super\DAG\nodag


In [ ]:
# Collider
import numpy as np

# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5


print("R_hat = \n",np.array2string(R_hat, formatter={'float_kind':lambda x: f"{x:.2f}"}))
A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)

(3, 3)
R_hat = 
 [[1.01 1.02 -0.01]
 [1.02 3.08 1.02]
 [-0.01 1.02 1.00]]
A_true = 
 [[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]


In [14]:
# A->B<-C
B_true = np.array([
    [0, 1, 0],
    [0, 0, 0],
    [0, 1, 0]
])

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, seed=42)
R_hat = np.cov(data.T)
print("R_hat= \n",np.array2string(R_hat, formatter={'float_kind':lambda x: f"{x:.2f}"}))
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true= \n",np.array2string(A_true, formatter={'float_kind':lambda x: f"{x:.0f}"}))
print(np.trace(A_true.T @ R_hat @ A_true))
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)

print("likelihood_true = ", likelihood_true)
print("-2logdet = ",-2 * np.log(np.linalg.det(A_true)))
print("trace = ",np.trace(A_true.T @ R_hat @ A_true))

t = 1
tol = 0.005      
lam = 0.5         

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed,
        printreset=False
    )
    print("B_final = \n", np.array2string(B_final, formatter={'float_kind':lambda x: f"{x:.2f}"}))
    print("G_final = \n", G_final)
    print(- 2 * np.log(np.linalg.det(B_final)) + np.trace(B_final.T @ R_hat @ B_final))

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([
            [0, 1, 0],
            [0, 0, 0],
            [0, 1, 0]
            ]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    print("-2logdet = ",-2 * np.log(np.linalg.det(B_final)))
    print("trace = ",np.trace(B_final.T @ R_hat @ B_final))
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

R_hat= 
 [[1.02 1.03 0.01]
 [1.03 3.08 1.03]
 [0.01 1.03 1.01]]
A_true= 
 [[1 -1 0]
 [0 1 0]
 [0 -1 1]]
3.024905302719005
likelihood_true =  3.024905302719005
-2logdet =  -0.0
trace =  3.024905302719005
B_final = 
 [[1.23 -0.00 0.72]
 [-0.82 0.57 0.00]
 [1.23 0.00 -0.71]]
G_final = 
 [[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
nan
[00] likelihood_est=3.0248, diff_ratio=0.0000 -> ✅
-2logdet =  nan
trace =  3.0260745683807473
penalty = 1.4910

=== Summary ===
Total runs: 1
True times: 0
True rate: 0.0
Tolerance (relative error): 0.01
Converged runs: 1
Convergence ratio: 1.000
Average diff ratio: 0.0000
Median diff ratio: 0.0000


C:\Users\super\AppData\Local\Temp\ipykernel_16748\2329800557.py:48: RuntimeWarning: invalid value encountered in log
  print(- 2 * np.log(np.linalg.det(B_final)) + np.trace(B_final.T @ R_hat @ B_final))
C:\Users\super\AppData\Local\Temp\ipykernel_16748\2329800557.py:69: RuntimeWarning: invalid value encountered in log
  print("-2logdet = ",-2 * np.log(np.linalg.det(B_final)))


In [ ]:
# A->B->C D
B_true = np.array([
    [0, 1, 0, 0],
    [0, 0, 2, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, seed=42)
R_hat = np.cov(data.T)
print("R_hat= \n",np.array2string(R_hat, formatter={'float_kind':lambda x: f"{x:.2f}"}))
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true= \n",np.array2string(A_true, formatter={'float_kind':lambda x: f"{x:.0f}"}))
print(np.trace(A_true.T @ R_hat @ A_true))
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
print("-2logdet = ",-2 * np.log(np.linalg.det(A_true)))
print("trace = ",np.trace(A_true.T @ R_hat @ A_true))

t = 1
tol = 0.005      
lam = 0.5         

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed,
        printreset=False
    )
    print("B_final = \n", np.array2string(B_final, formatter={'float_kind':lambda x: f"{x:.2f}"}))
    print("G_final = \n", G_final)
    print(- 2 * np.log(np.linalg.det(B_final)) + np.trace(B_final.T @ R_hat @ B_final))

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0]
        ]))
        or np.array_equal(G_final, np.array([
            [0, 0, 0, 0],
            [1, 0, 1, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0]
        ]))
        or np.array_equal(G_final, np.array([
            [0, 0, 0, 0],
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 0]
        ]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    print("-2logdet = ",-2 * np.log(np.linalg.det(B_final)))
    print("trace = ",np.trace(B_final.T @ R_hat @ B_final))
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

R_hat= 
 [[1.00 1.00 2.00 0.00]
 [1.00 2.00 4.00 0.00]
 [2.00 4.00 9.00 0.00]
 [0.00 0.00 0.00 1.00]]
A_true= 
 [[1 -1 0 0]
 [0 1 -2 0]
 [0 0 1 0]
 [0 0 0 1]]
3.9990966386111384
likelihood_true =  3.9990966386111384
-2logdet =  -0.0
trace =  3.9990966386111384
B_final = 
 [[-0.00 0.00 1.35 -0.00]
 [-0.00 -2.13 -0.00 0.00]
 [-0.00 0.95 -0.29 0.33]
 [1.02 -0.00 -0.00 -0.00]]
G_final = 
 [[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 1.]
 [1. 0. 0. 0.]]
nan
[00] likelihood_est=4.1035, diff_ratio=0.0261 -> ❌
-2logdet =  nan
trace =  4.0506864824925595
penalty = 2.0040

=== Summary ===
Total runs: 1
True times: 0
True rate: 0.0
Tolerance (relative error): 0.01
Converged runs: 0
Convergence ratio: 0.000
Average diff ratio: 0.0261
Median diff ratio: 0.0261


C:\Users\super\AppData\Local\Temp\ipykernel_16748\2745374160.py:48: RuntimeWarning: invalid value encountered in log
  print(- 2 * np.log(np.linalg.det(B_final)) + np.trace(B_final.T @ R_hat @ B_final))
C:\Users\super\AppData\Local\Temp\ipykernel_16748\2745374160.py:82: RuntimeWarning: invalid value encountered in log
  print("-2logdet = ",-2 * np.log(np.linalg.det(B_final)))


In [17]:
# A->B->C D
B_true = np.array([
    [0, 1, 0, 0],
    [0, 0, 2, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])

data, G, B, Sigma = generate_scm_from_BN(B_true.T, n_samples=10000, seed=42)
R_hat = np.cov(data.T)
print("R_hat= \n",np.array2string(R_hat, formatter={'float_kind':lambda x: f"{x:.2f}"}))
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true= \n",np.array2string(A_true, formatter={'float_kind':lambda x: f"{x:.0f}"}))
print(np.trace(A_true.T @ R_hat @ A_true))
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
print("-2logdet = ",-2 * np.log(np.linalg.det(A_true)))
print("trace = ",np.trace(A_true.T @ R_hat @ A_true))

t = 100
tol = 0.005      
lam = 0.5         
iters = [100, 500, 1000, 2000, 3000, 5000, 7500, 10000, 30000]
for iter in iters:
    print(f"ITER = {iter}")

    likelihood_diffs = []
    success_count = 0
    true_count = 0
    for seed in range(t):
        np.random.seed(seed)
        B_init = np.random.randn(*R_hat.shape)

        B_final, G_final, info = train_gs_reset(
            Rhat_np=R_hat,
            lam=lam,
            delta=1e-6,
            tau_start=1,
            tau_end=1,
            B_init=B_init,
            max_steps=iter,
            history_every=100,
            seed=seed,
            printreset=False
        )
        #print("B_final = \n", np.array2string(B_final, formatter={'float_kind':lambda x: f"{x:.2f}"}))
        #print("G_final = \n", G_final)
        #print(- 2 * np.log(np.linalg.det(B_final)) + np.trace(B_final.T @ R_hat @ B_final))

        likelihood_est = info["final_likelihood"]
        penalty_est = info["final_penalty"]

        diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
        likelihood_diffs.append(diff_ratio)
        if (
            np.array_equal(G_final, np.array([
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]
            ]))
            or np.array_equal(G_final, np.array([
                [0, 0, 0, 0],
                [1, 0, 1, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]
            ]))
            or np.array_equal(G_final, np.array([
                [0, 0, 0, 0],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 0]
            ]))
            ):  
            true_count += 1
            print("True result, seed = ",seed)
        is_converged = diff_ratio < tol
        success_count += int(is_converged)
        status = "✅" if is_converged else "❌"
        #print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
        #f"diff_ratio={diff_ratio:.4f} -> {status}")
        #print("-2logdet = ",-2 * np.log(np.linalg.det(B_final)))
        #print("trace = ",np.trace(B_final.T @ R_hat @ B_final))
        #print(f"penalty = {penalty_est:.4f}")



    eps = 1e-6
    abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

    # 计算 U_init = logit(|B_init|)
    U_init = np.log(abs_B / (1 - abs_B))

    # 计算 P_init = sign(B_init) * |B_init|
    P_init = np.sign(B_init) * abs_B




    likelihood_diffs = np.array(likelihood_diffs)
    convergence_ratio = success_count / t
    print("\n=== Summary ===")
    print(f"Total runs: {t}")
    print("True times:", true_count)
    print(f"Converged runs: {success_count}\n\n")

R_hat= 
 [[0.98 0.99 1.98 0.00]
 [0.99 2.01 4.04 -0.01]
 [1.98 4.04 9.11 -0.02]
 [0.00 -0.01 -0.02 1.04]]
A_true= 
 [[1 -1 0 0]
 [0 1 -2 0]
 [0 0 1 0]
 [0 0 0 1]]
4.036124638121176
likelihood_true =  4.036124638121176
-2logdet =  -0.0
trace =  4.036124638121176
ITER = 100

=== Summary ===
Total runs: 100
True times: 0
Converged runs: 6


ITER = 500
True result, seed =  1
True result, seed =  24

=== Summary ===
Total runs: 100
True times: 2
Converged runs: 22


ITER = 1000
True result, seed =  1
True result, seed =  16
True result, seed =  24
True result, seed =  30
True result, seed =  58

=== Summary ===
Total runs: 100
True times: 5
Converged runs: 40


ITER = 2000
True result, seed =  1
True result, seed =  16
True result, seed =  24
True result, seed =  30
True result, seed =  58

=== Summary ===
Total runs: 100
True times: 5
Converged runs: 43


ITER = 3000
True result, seed =  1
True result, seed =  16
True result, seed =  24
True result, seed =  30
True result, seed =  58

=== 